In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import os
import torch.nn.functional as F

In [2]:
import torchvision.models as models

In [3]:
transforms = transforms.transforms.Compose([transforms.Resize([224,224]),transforms.ToTensor()])

In [5]:
data_dir='D:/ML/invasive/train'

image_datasets = datasets.ImageFolder(data_dir,transform=transforms)
train_set, val_set = torch.utils.data.random_split(image_datasets, [int(len(image_datasets)*0.8), int(len(image_datasets)*0.2)])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(val_set, batch_size=64, shuffle=False)

In [6]:
image_datasets.classes

['invasive', 'not-invasive']

In [7]:
"""import pandas as pd
import shutil
labels = pd.read_csv("D:/ML/invasive/train_labels.csv")"""

'import pandas as pd\nimport shutil\nlabels = pd.read_csv("D:/ML/invasive/train_labels.csv")'

In [8]:
"""for i in labels.iterrows():
    file_name = str(i[1]['name'])+'.jpg'
    if i[1]['invasive'] == 1:
        shutil.move(data_dir+"/train/"+file_name,data_dir+"/train/"+"invasive/"+file_name)
    else:
        shutil.move(data_dir+"/train/"+file_name,data_dir+"/train/"+"not-invasive/"+file_name)
    print(i[1]['name'],i[1]['invasive'])"""

'for i in labels.iterrows():\n    file_name = str(i[1][\'name\'])+\'.jpg\'\n    if i[1][\'invasive\'] == 1:\n        shutil.move(data_dir+"/train/"+file_name,data_dir+"/train/"+"invasive/"+file_name)\n    else:\n        shutil.move(data_dir+"/train/"+file_name,data_dir+"/train/"+"not-invasive/"+file_name)\n    print(i[1][\'name\'],i[1][\'invasive\'])'

In [41]:
def create_body():
    "Cut off the body of a typically pretrained `model` at `cut` (int) or cut the model as specified by `cut(model)` (function)."
    model = models.resnet34(pretrained=True)
    return nn.Sequential(*list(model.children())[:-1])


In [71]:
class conv_net(nn.Module):
    def __init__(self):
        super().__init__()

        self.body = create_body()

        conv_out_size = self._get_conv_out([3,224,224])
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
            nn.LogSoftmax()
        )

    def _get_conv_out(self, shape):
        o = self.body(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.body(x).view(x.size()[0], -1)
        return self.fc(conv_out)

In [72]:
model = conv_net()

In [73]:
#list(model.children())

In [74]:
n_layers = len(list(model.parameters()))-6
for layer in model.parameters():
    layer.requires_grad=False
    n_layers -=1
    if n_layers==0:
        break

In [75]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [76]:
if torch.cuda.is_available():
        device = torch.device("cuda")
else:
    device = torch.device("cpu")

model.to(device)

conv_net(
  (body): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 

In [80]:

epochs = 15
print_every = 50
for e in range(epochs):
    running_loss = 0
    for step, (x,y) in enumerate(train_loader):
        y = y.to(device)
        x = x.to(device)
        
        optimizer.zero_grad()
        
        # Forward and backward passes
        output = model.forward(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        

    print("Epoch: {}/{}... ".format(e+1, epochs),
        "Loss: {:.4f}".format(running_loss/print_every))

Epoch: 1/15...  Loss: 0.2515
Epoch: 2/15...  Loss: 0.1349
Epoch: 3/15...  Loss: 0.1604
Epoch: 4/15...  Loss: 0.1118
Epoch: 5/15...  Loss: 0.1034
Epoch: 6/15...  Loss: 0.1010
Epoch: 7/15...  Loss: 0.0785
Epoch: 8/15...  Loss: 0.0891
Epoch: 9/15...  Loss: 0.0810
Epoch: 10/15...  Loss: 0.0707
Epoch: 11/15...  Loss: 0.0767
Epoch: 12/15...  Loss: 0.0907
Epoch: 13/15...  Loss: 0.0726
Epoch: 14/15...  Loss: 0.0551
Epoch: 15/15...  Loss: 0.0674


In [82]:
test_loss = 0
correct = 0
preds_c = []
with torch.no_grad():
    for data, target in valid_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        preds_c.append(pred)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(valid_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(valid_loader.dataset),
    100. * correct / len(valid_loader.dataset)))

C:\Users\Paul\.conda\envs\universe\lib\site-packages\torch\nn\modules\container.py:100: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)



Test set: Average loss: 0.3123, Accuracy: 403/459 (88%)



In [ ]:
"""if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser(description='Train invasive species model.')
    parser.add_argument('architecture', type=str)
    parser.add_argument('--sum', dest='accumulate', action='store_const',
                        const=sum, default=max,
                        help='sum the integers (default: find the max)')

    args = parser.parse_args()
    print(args.accumulate(args.integers))"""